In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import transformer
from inference_model import TransformerNet, process_molecule
import numpy as np
import pandas as pd
import matplotlib as pyplot

In [4]:
import torch

In [5]:
model = TransformerNet.load_from_checkpoint("/home/dionizije/Documents/drug_attrition_oracle/src/smiles_only/epoch=6-step=405.ckpt")

In [6]:
model.state_dict()['model.encoder.layers.0.self_attn.linears.0.weight']

tensor([[ 0.0524,  0.0188,  0.0025,  ...,  0.0179,  0.0422, -0.0315],
        [ 0.0042,  0.0465, -0.0372,  ...,  0.0013, -0.0136,  0.0032],
        [ 0.0415,  0.0446,  0.0333,  ...,  0.0162, -0.0290, -0.0396],
        ...,
        [ 0.0379,  0.0147,  0.0354,  ...,  0.0410, -0.0403,  0.0409],
        [ 0.0473,  0.0434,  0.0340,  ...,  0.0197,  0.0395, -0.0343],
        [-0.0024, -0.0491, -0.0139,  ..., -0.0461,  0.0444, -0.0049]])

In [8]:
embed_wt = model.state_dict()["model.src_embed.lut.weight"]
embed_bias = model.state_dict()["model.src_embed.lut.bias"]

In [42]:
model.state_dict()

OrderedDict([('model.encoder.layers.0.self_attn.linears.0.weight',
              tensor([[ 0.0524,  0.0188,  0.0025,  ...,  0.0179,  0.0422, -0.0315],
                      [ 0.0042,  0.0465, -0.0372,  ...,  0.0013, -0.0136,  0.0032],
                      [ 0.0415,  0.0446,  0.0333,  ...,  0.0162, -0.0290, -0.0396],
                      ...,
                      [ 0.0379,  0.0147,  0.0354,  ...,  0.0410, -0.0403,  0.0409],
                      [ 0.0473,  0.0434,  0.0340,  ...,  0.0197,  0.0395, -0.0343],
                      [-0.0024, -0.0491, -0.0139,  ..., -0.0461,  0.0444, -0.0049]])),
             ('model.encoder.layers.0.self_attn.linears.0.bias',
              tensor([-0.0195, -0.0011,  0.0253,  ...,  0.0225, -0.0107, -0.0001])),
             ('model.encoder.layers.0.self_attn.linears.1.weight',
              tensor([[-0.0020, -0.0535, -0.0493,  ...,  0.0275,  0.0357, -0.0167],
                      [ 0.0266,  0.0223, -0.0377,  ...,  0.0374, -0.0447, -0.0442],
              

In [9]:
attn = model.state_dict()['model.encoder.layers.0.self_attn.linears.0.weight']

In [24]:
node, adj, dist, mask = process_molecule("C1=CC=CC=C1")
node = node.squeeze(0)
adj = adj.detach().cpu().numpy()
dist = dist.detach().cpu().numpy()
node = torch.transpose(node, 0, 0)

In [11]:
node.size()

torch.Size([7, 28])

In [12]:
embed_wt = torch.transpose(embed_wt, 1, 0)

In [13]:
embed_bias = torch.add(embed_bias, 1, 0)

/home/dionizije/anaconda3/envs/withdrawn/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This overload of add is deprecated:
	add(Tensor input, Number alpha, Tensor other, *, Tensor out)
Consider using one of the following signatures instead:
	add(Tensor input, Tensor other, *, Number alpha, Tensor out) (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448202689/work/torch/csrc/utils/python_arg_parser.cpp:1025.)
  """Entry point for launching an IPython kernel.


In [14]:
embed_bias.size()

torch.Size([1024])

In [15]:
embed_wt.size()

torch.Size([28, 1024])

In [16]:
embedding = torch.matmul(node, embed_wt)

In [17]:
embedding = torch.add(embedding, embed_bias)

In [18]:
atn_map = torch.matmul(embedding, attn).detach().cpu().numpy()

In [20]:
atn_map = pd.DataFrame(atn_map).transpose()